# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# Skip reading of checkpoints
    if 'checkpoints' not in root:
# join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root,'*.csv'))
#         print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create Apache Cassandra tavbles and queries.

## Located within the Workspace directory is the event_datafile_new.csv which contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    
# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Truncate Printout String for Easy Reading

In [8]:
# Truncate printout string
def truncate_printout(feature, feature_spacing):
    if len(feature) > feature_spacing: 
        feature = feature[:feature_spacing - 10] + '...'
    return feature

### Create tables and query

#### 1. Create query to return the artist, song title and song's length in the music app history that was heard during a specifc sessionId and specific itemInSession (e.g. sessionId = 338, and itemInSession  = 4)

##### Expected output: "Name of the artist, title of the song and length of the track"
##### Based on: "sessionId and itemInSession"

##### Composite Key: "sessionid, itemInSession"

In [9]:
# Create table to query artist, song title and song's length for each sessionid and itemInSession.
query = "CREATE TABLE IF NOT EXISTS music_library_by_session "
query = query + "(sessionid int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionid, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [10]:
# Read in CSV and populate table for each input.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Populate table with sessionid, itemInSession, artist, song, length
        query = "INSERT INTO music_library_by_session (sessionid, iteminsession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign corresponding column elements.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
# Test to verify data populated.
query = "select artist, song, length from music_library_by_session WHERE sessionid=338 and iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Print out test results
artist_spacing = 30
song_spacing = 30
length_spacing = 10
template = '{0:' + str(artist_spacing) + '}| {1:' + str(song_spacing) + '}| {2:' + str(length_spacing) + '}'
print(template.format('Artist Name', 'Song Title', 'Song Length'))
print('-' * (artist_spacing + song_spacing + length_spacing))
for row in rows:  
    artist = truncate_printout(row.artist, artist_spacing)
    song = truncate_printout(row.song, song_spacing)
    
    print(template.format(artist, song, row.length))

Artist Name                   | Song Title                    | Song Length
----------------------------------------------------------------------
Faithless                     | Music Matters (Mark ...       | 495.30731201171875


#### 2. Create query to return the name of artist, song (sorted by itemInSession) and user (first and last name) for specific userid and specific sessionid (e.g. userid = 10, sessionid = 182)

##### Expected output: "Name of the artist, title of the song (sorted by itemInSession) and user (first and last name)"
##### Based on: "userid and sessionid"

##### Composite Key: "userid, sessionid"
##### Clustering Key: "itemInSession" (added for uniqueness and sorting)

In [12]:
# Create table to query artist, song title and user first and last name for each userid and sessionid.
query = "CREATE TABLE IF NOT EXISTS music_library_by_user_session "
query = query + "(userid int, sessionid int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userid, sessionid), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [13]:
# Read in CSV and populate table for each input.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Populate table with userid, sessionid, artist, song, firstName, lastName
        query = "INSERT INTO music_library_by_user_session (userid, sessionid, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign corresponding column elements.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
# Test to verify data populated.
query = "select itemInSession, artist, song, firstName, lastName from music_library_by_user_session WHERE userid=10 and sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Print out test results
iteminsession_spacing = 10
artist_spacing = 30
song_spacing = 30
firstname_spacing = 20
lastname_spacing = 20
template = '{0:' + str(iteminsession_spacing) + '}| {1:' + str(artist_spacing) + '}| {2:' + str(song_spacing) + '}| {3:' + str(firstname_spacing) + '}| {4:' + str(lastname_spacing) + '}'
print(template.format('Item', 'Artist Name', 'Song Title', 'User First Name', 'User Last Name' ))
print('-' * (iteminsession_spacing + artist_spacing + song_spacing + firstname_spacing + lastname_spacing))
for row in rows:
    artist = truncate_printout(row.artist, artist_spacing)
    song = truncate_printout(row.song, song_spacing)
    fist_name = truncate_printout(row.firstname, firstname_spacing)
    last_name = truncate_printout(row.lastname, lastname_spacing)
    
    print(template.format(str(row.iteminsession), artist, song, fist_name, last_name))

Item      | Artist Name                   | Song Title                    | User First Name     | User Last Name      
--------------------------------------------------------------------------------------------------------------
0         | Down To The Bone              | Keep On Keepin' On            | Sylvie              | Cruz                
1         | Three Drives                  | Greece 2000                   | Sylvie              | Cruz                
2         | Sebastien Tellier             | Kilometer                     | Sylvie              | Cruz                
3         | Lonnie Gordon                 | Catch You Baby (Stev...       | Sylvie              | Cruz                


#### 3. Create query to return every user name (first and last) (sorted by userid) who listened to a specific song (e.g. song ='All Hands Against His Own')

##### Expected output: "user (first and last name)"
##### Based on: "title of song"

##### Composite Key: "song"
##### Clustering Key: "userid" (added for uniqueness and sorting)

In [15]:
# Create table to query user first and last name for each song.
query = "CREATE TABLE IF NOT EXISTS music_library_by_songs_played "
query = query + "(song text, userid int, firstName text, lastName text, PRIMARY KEY ((song), userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)    
                    

In [16]:
# Read in CSV and populate table for each input.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Populate table with userid, sessionid, artist, song, firstName, lastName
        query = "INSERT INTO music_library_by_songs_played (song, userid, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign corresponding column elements.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
# Test to verify data populated.
query = "select userid, firstName, lastName from music_library_by_songs_played WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Print out test results
userid_spacing = 10
firstname_spacing = 20
lastname_spacing = 20
template = '{0:' + str(userid_spacing) + '} | {1:' + str(firstname_spacing) + '} | {2:' + str(lastname_spacing) + '}'
print(template.format('User Id', 'User First Name', 'User Last Name' ))
print('-' * (userid_spacing + firstname_spacing + lastname_spacing))
for row in rows:
    fist_name = truncate_printout(row.firstname, firstname_spacing)
    last_name = truncate_printout(row.lastname, lastname_spacing)
    
    print(template.format(str(row.userid), fist_name, last_name))

User Id    | User First Name      | User Last Name      
--------------------------------------------------
29         | Jacqueline           | Lynch               
80         | Tegan                | Levine              
95         | Sara                 | Johnson             


#### Drop the tables before closing out the sessions

In [18]:
# Drop tables created
query = "drop table if exists music_library_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists music_library_by_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists music_library_by_songs_played"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()